# Import libraries

In [ ]:
!pip install category_encoders
import pandas as pd
import io
import requests
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import random
from category_encoders import MEstimateEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.svm import SVC


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Work on dataset


Download dataset

In [ ]:
# Download the name of columns.
header=requests.get("https://cloudstor.aarnet.edu.au/plus/s/umT99TnxvbpkkoE/download?path=%2FCSV%2FEntire%20Dataset&files=UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv").content
header=pd.read_csv(io.StringIO(header.decode('utf-8')), low_memory=False)
columns = header.columns
columns

Index(['pkSeqID', 'stime', 'flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport',
       'pkts', 'bytes', 'state', 'ltime', 'seq', 'dur', 'mean', 'stddev',
       'smac', 'dmac', 'sum', 'min', 'max', 'soui', 'doui', 'sco', 'dco',
       'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate',
       'attack', 'category', 'subcategory '],
      dtype='object')

In [ ]:
# Download the dataset.
# The urls are csv files each one contain 1M row.
data1=requests.get("https://cloudstor.aarnet.edu.au/plus/s/umT99TnxvbpkkoE/download?path=%2FCSV%2FEntire%20Dataset&files=UNSW_2018_IoT_Botnet_Dataset_1.csv").content
data2=requests.get("https://cloudstor.aarnet.edu.au/plus/s/umT99TnxvbpkkoE/download?path=%2FCSV%2FEntire%20Dataset&files=UNSW_2018_IoT_Botnet_Dataset_2.csv").content

data1=pd.read_csv(io.StringIO(data1.decode('utf-8')), low_memory=False, header = None)
data2=pd.read_csv(io.StringIO(data2.decode('utf-8')), low_memory=False, header = None)

Reading dataset

In [ ]:
# Save the dataset to local.

# First csv.
data1.to_csv(
    "data1.csv",
    header=columns,
    index=False
    )
print("finish save csv number = " + str(1))


# Second csv.
data2.to_csv(
    "data2.csv",
    header=columns,
    index=False
    )
print("finish save csv number = " + str(2))

finish save csv number = 1
finish save csv number = 2


In [ ]:
# Read and print the first csv. 
data_part1 = pd.read_csv("data1.csv", low_memory = False)
data_part1

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,1,1.526344e+09,e,arp,192.168.100.1,NaN,192.168.100.3,NaN,4,240,...,2,2,120,120,0.002508,0.000836,0.000836,0,Normal,Normal
1,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
2,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
3,4,1.526344e+09,e,arp,192.168.100.4,NaN,192.168.100.7,NaN,10,510,...,5,5,210,300,0.006189,0.002751,0.002751,0,Normal,Normal
4,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999996,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,12020,2,120,...,1,1,60,60,23.854963,0.000000,0.000000,1,Reconnaissance,Service_Scan
999996,999997,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,60532,2,120,...,1,1,60,60,23.854963,0.000000,0.000000,1,Reconnaissance,Service_Scan
999997,999998,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,52144,2,120,...,1,1,60,60,23.862362,0.000000,0.000000,1,Reconnaissance,Service_Scan
999998,999999,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,58105,2,120,...,1,1,60,60,23.854963,0.000000,0.000000,1,Reconnaissance,Service_Scan


In [ ]:
# Read and print the second csv. 
data_part2 = pd.read_csv("data2.csv", low_memory = False)
data_part2

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,1000001,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,41081,2,120,...,1,1,60,60,23.862932,0.00000,0.0,1,Reconnaissance,Service_Scan
1,1000002,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,13371,2,120,...,1,1,60,60,23.868057,0.00000,0.0,1,Reconnaissance,Service_Scan
2,1000003,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,3796,2,120,...,1,1,60,60,23.868057,0.00000,0.0,1,Reconnaissance,Service_Scan
3,1000004,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,63993,2,120,...,1,1,60,60,23.860085,0.00000,0.0,1,Reconnaissance,Service_Scan
4,1000005,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,27356,2,120,...,1,1,60,60,23.868628,0.00000,0.0,1,Reconnaissance,Service_Scan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1999996,1.528081e+09,e s,tcp,192.168.100.149,30110,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.10977,0.0,1,DoS,TCP
999996,1999997,1.528081e+09,e s,tcp,192.168.100.149,30111,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.10977,0.0,1,DoS,TCP
999997,1999998,1.528081e+09,e s,tcp,192.168.100.149,30112,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.10977,0.0,1,DoS,TCP
999998,1999999,1.528081e+09,e s,tcp,192.168.100.149,30113,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.10977,0.0,1,DoS,TCP


In [ ]:
# Merge the two csv in one csv then print it.
merged = pd.concat(
    map(
        pd.read_csv, 
                     [
                      "data1.csv",
                      "data2.csv"
                     ] , 
       )
          , ignore_index=True
                              )
merged

/usr/local/lib/python3.8/dist-packages/pandas/core/reshape/concat.py:294: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(


,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,1,1.526344e+09,e,arp,192.168.100.1,NaN,192.168.100.3,NaN,4,240,...,2,2,120,120,0.002508,0.000836,0.000836,0,Normal,Normal
1,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
2,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
3,4,1.526344e+09,e,arp,192.168.100.4,NaN,192.168.100.7,NaN,10,510,...,5,5,210,300,0.006189,0.002751,0.002751,0,Normal,Normal
4,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,1999996,1.528081e+09,e s,tcp,192.168.100.149,30110,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999996,1999997,1.528081e+09,e s,tcp,192.168.100.149,30111,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999997,1999998,1.528081e+09,e s,tcp,192.168.100.149,30112,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999998,1999999,1.528081e+09,e s,tcp,192.168.100.149,30113,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP


In [ ]:
# Save the merged csv to local.
merged.to_csv("merged.csv", index=False)

In [ ]:
# Read and print the merged csv.
data = pd.read_csv("merged.csv", low_memory = False)
data

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,1,1.526344e+09,e,arp,192.168.100.1,NaN,192.168.100.3,NaN,4,240,...,2,2,120,120,0.002508,0.000836,0.000836,0,Normal,Normal
1,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
2,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
3,4,1.526344e+09,e,arp,192.168.100.4,NaN,192.168.100.7,NaN,10,510,...,5,5,210,300,0.006189,0.002751,0.002751,0,Normal,Normal
4,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,1999996,1.528081e+09,e s,tcp,192.168.100.149,30110,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999996,1999997,1.528081e+09,e s,tcp,192.168.100.149,30111,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999997,1999998,1.528081e+09,e s,tcp,192.168.100.149,30112,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999998,1999999,1.528081e+09,e s,tcp,192.168.100.149,30113,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP


Processing dataset

In [ ]:
# Check if there is any null value.
data.isnull().sum()

pkSeqID               0
stime                 0
flgs                  0
proto                 0
saddr                 0
sport               877
daddr                 0
dport               877
pkts                  0
bytes                 0
state                 0
ltime                 0
seq                   0
dur                   0
mean                  0
stddev                0
smac            2000000
dmac            2000000
sum                   0
min                   0
max                   0
soui            2000000
doui            2000000
sco             2000000
dco             2000000
spkts                 0
dpkts                 0
sbytes                0
dbytes                0
rate                  0
srate                 0
drate                 0
attack                0
category              0
subcategory           0
dtype: int64

In [ ]:
# From the result: 
# we can see that there are columns which all of there values are null so we have to drop them.
# and some columns have some null values so we have to drop the rows that have the null value.

# Drop columns.
data = data.drop(columns= ["smac", "dmac", "soui", "doui", "sco", "dco"])
# Drop rows.
data = data.dropna()

In [ ]:
# Checking for null values after dropping.
data.isnull().sum()

pkSeqID         0
stime           0
flgs            0
proto           0
saddr           0
sport           0
daddr           0
dport           0
pkts            0
bytes           0
state           0
ltime           0
seq             0
dur             0
mean            0
stddev          0
sum             0
min             0
max             0
spkts           0
dpkts           0
sbytes          0
dbytes          0
rate            0
srate           0
drate           0
attack          0
category        0
subcategory     0
dtype: int64

In [ ]:
# Printing dataset.
data

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
1,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
2,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
4,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal
7,8,1.526344e+09,e,udp,192.168.100.150,58360,192.168.217.2,53,2,172,...,2,0,172,0,0.399984,0.399984,0.000000,0,Normal,Normal
8,9,1.526344e+09,e,udp,192.168.100.149,37214,192.168.217.2,53,2,172,...,2,0,172,0,0.399824,0.399824,0.000000,0,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,1999996,1.528081e+09,e s,tcp,192.168.100.149,30110,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999996,1999997,1.528081e+09,e s,tcp,192.168.100.149,30111,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999997,1999998,1.528081e+09,e s,tcp,192.168.100.149,30112,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP
1999998,1999999,1.528081e+09,e s,tcp,192.168.100.149,30113,192.168.100.5,80,4,616,...,4,0,616,0,0.109770,0.109770,0.000000,1,DoS,TCP


In [ ]:
# In target column (attack column) it has two classes, if there is an attack (represented by number 1) or if there is no attack (represented by number 0)
# So we print the count for each class.
count_0 = 0
count_1 = 0
for i in data["attack"]:
  if i == 0:
    count_0+=1
  if i == 1:
    count_1+=1

print("zeros", count_0)
print("ones", count_1)

zeros 6542
ones 1992581


In [ ]:
# Taking all rows of class 0.
# Taking class 1 rows with size of class 0 rows + 4000.
data_zeros = data.query('attack == 0', inplace=False)
data_ones = data.query('attack == 1', inplace=False)
data_ones_less = data_ones.iloc[0 : data_zeros["attack"].size + 4000]


In [ ]:
# Save the class 0 rows to local for merging.
data_zeros.to_csv(
    "data_zeros.csv",
    index=False
    )

# Save the class 1 rows to local for merging.
data_ones_less.to_csv(
    "data_ones_less.csv",
    index=False
    )

In [ ]:
# Merging the class 0 rows and class 1 rows 
data_final = pd.concat(
    map(
        pd.read_csv, 
                     [
                      "data_zeros.csv",
                      "data_ones_less.csv"
                     ] , 
       )
          , ignore_index=True
                              )
data_final

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
1,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
2,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal
3,8,1.526344e+09,e,udp,192.168.100.150,58360,192.168.217.2,53,2,172,...,2,0,172,0,0.399984,0.399984,0.000000,0,Normal,Normal
4,9,1.526344e+09,e,udp,192.168.100.149,37214,192.168.217.2,53,2,172,...,2,0,172,0,0.399824,0.399824,0.000000,0,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17079,10608,1.526345e+09,eU,udp,192.168.100.148,35008,192.168.100.3,32770,1,60,...,1,0,60,0,0.000000,0.000000,0.000000,1,Reconnaissance,Service_Scan
17080,10609,1.526345e+09,e,icmp,192.168.100.3,0x0303,192.168.100.148,0x0280,1,70,...,1,0,70,0,0.000000,0.000000,0.000000,1,Reconnaissance,Service_Scan
17081,10610,1.526345e+09,eU,udp,192.168.100.149,39704,192.168.100.3,47808,1,60,...,1,0,60,0,0.000000,0.000000,0.000000,1,Reconnaissance,Service_Scan
17082,10611,1.526345e+09,e,icmp,192.168.100.3,0x0303,192.168.100.149,0xc0ba,1,70,...,1,0,70,0,0.000000,0.000000,0.000000,1,Reconnaissance,Service_Scan


In [ ]:
data_final.columns   

Index(['pkSeqID', 'stime', 'flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport',
       'pkts', 'bytes', 'state', 'ltime', 'seq', 'dur', 'mean', 'stddev',
       'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate',
       'srate', 'drate', 'attack', 'category', 'subcategory '],
      dtype='object')

proseccing

In [ ]:
# Taking the attack column form dataset because this column is the label for training.
y = data_final["attack"]

# dropping the (pkSeqID, category, subcategory, attack) columns because:
# pkSeqID is an index column.
# category and subcategory are another representation for label.
# attack is the label column.
data_final = data_final.drop(columns = ["pkSeqID", "category", "subcategory ", "attack"])


In [ ]:
# Taking columns which the type of them is object to encode them.
data_final_cat = data_final[[column for column in data_final.columns if data_final[column].dtype == "object"]]

In [ ]:
# Print the columns of type object.
data_final_cat.dtypes

flgs     object
proto    object
saddr    object
sport    object
daddr    object
dport    object
state    object
dtype: object

In [ ]:
# Encoding each column alone by using MEstimateEncoder and then merge them.
data_final_enc_cat = pd.DataFrame() 
d = data_final_cat.copy()

for column in data_final_cat:
  ME = MEstimateEncoder(m=2)
  d_ME = ME.fit_transform(d[column], y)
  data_final_enc_cat = pd.concat([data_final_enc_cat, d_ME], axis = 1)


In [ ]:
# Print the encoded columns.
data_final_enc_cat

,flgs,proto,saddr,sport,daddr,dport,state
0,0.544497,0.919891,0.000494,0.176305,0.137126,0.176305,0.002805
1,0.544497,0.387977,0.778396,0.411379,0.068563,0.018164,0.002805
2,0.544497,0.387977,0.017382,0.051422,0.017382,0.003164,0.002805
3,0.544497,0.387977,0.779011,0.411379,0.002550,0.003164,0.530947
4,0.544497,0.387977,0.778396,0.411379,0.002550,0.003164,0.530947
...,...,...,...,...,...,...,...
17079,0.996548,0.387977,0.781519,0.997292,0.981328,0.923414,0.530947
17080,0.544497,0.996242,0.690758,0.996242,0.998885,0.846827,0.996242
17081,0.996548,0.387977,0.778396,0.998824,0.981328,0.808534,0.530947
17082,0.544497,0.996242,0.690758,0.996242,0.998853,0.808534,0.996242


In [ ]:
# Taking columns which the type of them is not object to scale them.
data_final_numr = data_final[[column for column in data_final.columns if data_final[column].dtype != "object"]]

In [ ]:
# Print the columns and there type.
data_final_numr.dtypes

stime     float64
pkts        int64
bytes       int64
ltime     float64
seq         int64
dur       float64
mean      float64
stddev    float64
sum       float64
min       float64
max       float64
spkts       int64
dpkts       int64
sbytes      int64
dbytes      int64
rate      float64
srate     float64
drate     float64
dtype: object

In [ ]:
# Scaling the columns using StandardScaler.
SS = StandardScaler()
data_final_enc_numr = SS.fit_transform(data_final_numr.to_numpy())
data_final_enc_numr = pd.DataFrame(data_final_enc_numr, columns = data_final_numr.columns)

# # Print the scaled columns.
data_final_enc_numr

,stime,pkts,bytes,ltime,seq,dur,mean,stddev,sum,min,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate
0,-0.539212,-0.095645,-0.088200,-0.535633,-0.419465,5.310773,-0.182877,-0.095458,-0.130870,-0.164911,-0.186437,-0.099767,-0.061523,-0.091283,-0.062695,-0.171246,-0.031934,-0.022898
1,-0.539203,-0.096671,-0.088268,-0.539248,-0.419432,-0.153460,-0.113565,-0.095536,-0.130656,-0.073611,-0.121169,-0.100493,-0.062774,-0.091335,-0.062771,-0.170890,-0.031934,-0.022909
2,-0.539014,-0.096414,-0.088207,-0.537639,-0.419335,1.988360,-0.042250,0.682195,-0.129998,-0.130428,0.053814,-0.100312,-0.062461,-0.091318,-0.062655,-0.171246,-0.031934,-0.022902
3,-0.538955,-0.096671,-0.088269,-0.538994,-0.419205,-0.144246,-0.182917,-0.095536,-0.130871,-0.164953,-0.186494,-0.100312,-0.063087,-0.091319,-0.062800,-0.171240,-0.031907,-0.022909
4,-0.538950,-0.096671,-0.088269,-0.538989,-0.418457,-0.144243,-0.182917,-0.095536,-0.130871,-0.164953,-0.186494,-0.100312,-0.063087,-0.091319,-0.062800,-0.171240,-0.031907,-0.022909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17079,-0.537128,-0.096799,-0.088284,-0.537173,-0.062730,-0.153642,-0.182829,-0.095536,-0.130871,-0.164838,-0.186412,-0.100493,-0.063087,-0.091341,-0.062800,-0.171247,-0.031934,-0.022909
17080,-0.537128,-0.096799,-0.088283,-0.537173,-0.062697,-0.153643,-0.182917,-0.095536,-0.130871,-0.164953,-0.186494,-0.100493,-0.063087,-0.091339,-0.062800,-0.171247,-0.031934,-0.022909
17081,-0.537127,-0.096799,-0.088284,-0.537173,-0.062665,-0.153642,-0.182867,-0.095536,-0.130871,-0.164887,-0.186447,-0.100493,-0.063087,-0.091341,-0.062800,-0.171247,-0.031934,-0.022909
17082,-0.537127,-0.096799,-0.088283,-0.537173,-0.062632,-0.153643,-0.182917,-0.095536,-0.130871,-0.164953,-0.186494,-0.100493,-0.063087,-0.091339,-0.062800,-0.171247,-0.031934,-0.022909


In [ ]:
# Merging the encoded columns and scaled columns to have the final dataset.
data_final_enc = pd.concat([data_final_enc_cat, data_final_enc_numr], axis = 1)

# Print the result of merging.
data_final_enc

,flgs,proto,saddr,sport,daddr,dport,state,stime,pkts,bytes,...,sum,min,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate
0,0.544497,0.919891,0.000494,0.176305,0.137126,0.176305,0.002805,-0.539212,-0.095645,-0.088200,...,-0.130870,-0.164911,-0.186437,-0.099767,-0.061523,-0.091283,-0.062695,-0.171246,-0.031934,-0.022898
1,0.544497,0.387977,0.778396,0.411379,0.068563,0.018164,0.002805,-0.539203,-0.096671,-0.088268,...,-0.130656,-0.073611,-0.121169,-0.100493,-0.062774,-0.091335,-0.062771,-0.170890,-0.031934,-0.022909
2,0.544497,0.387977,0.017382,0.051422,0.017382,0.003164,0.002805,-0.539014,-0.096414,-0.088207,...,-0.129998,-0.130428,0.053814,-0.100312,-0.062461,-0.091318,-0.062655,-0.171246,-0.031934,-0.022902
3,0.544497,0.387977,0.779011,0.411379,0.002550,0.003164,0.530947,-0.538955,-0.096671,-0.088269,...,-0.130871,-0.164953,-0.186494,-0.100312,-0.063087,-0.091319,-0.062800,-0.171240,-0.031907,-0.022909
4,0.544497,0.387977,0.778396,0.411379,0.002550,0.003164,0.530947,-0.538950,-0.096671,-0.088269,...,-0.130871,-0.164953,-0.186494,-0.100312,-0.063087,-0.091319,-0.062800,-0.171240,-0.031907,-0.022909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17079,0.996548,0.387977,0.781519,0.997292,0.981328,0.923414,0.530947,-0.537128,-0.096799,-0.088284,...,-0.130871,-0.164838,-0.186412,-0.100493,-0.063087,-0.091341,-0.062800,-0.171247,-0.031934,-0.022909
17080,0.544497,0.996242,0.690758,0.996242,0.998885,0.846827,0.996242,-0.537128,-0.096799,-0.088283,...,-0.130871,-0.164953,-0.186494,-0.100493,-0.063087,-0.091339,-0.062800,-0.171247,-0.031934,-0.022909
17081,0.996548,0.387977,0.778396,0.998824,0.981328,0.808534,0.530947,-0.537127,-0.096799,-0.088284,...,-0.130871,-0.164887,-0.186447,-0.100493,-0.063087,-0.091341,-0.062800,-0.171247,-0.031934,-0.022909
17082,0.544497,0.996242,0.690758,0.996242,0.998853,0.808534,0.996242,-0.537127,-0.096799,-0.088283,...,-0.130871,-0.164953,-0.186494,-0.100493,-0.063087,-0.091339,-0.062800,-0.171247,-0.031934,-0.022909


In [ ]:
# Print the label column.
y

0        0
1        0
2        0
3        0
4        0
        ..
17079    1
17080    1
17081    1
17082    1
17083    1
Name: attack, Length: 17084, dtype: int64

In [ ]:
# Shuffle the dataset and label with respect that each row on dataset and the opposite side of it on label is the same but in different location.
X, Y = shuffle(data_final_enc, y)

# Splite dataset in to train and test.
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

# Split train in to train and validation.
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

In [ ]:
print("shape of x_train", x_train.shape)
print("shape of y_train", y_train.shape)
print()
print("shape of x_val", x_val.shape)
print("shape of y_val", y_val.shape)
print()
print("shape of x_test", x_test.shape)
print("shape of y_test", y_test.shape)

shape of x_train (10933, 25)
shape of y_train (10933,)

shape of x_val (2734, 25)
shape of y_val (2734,)

shape of x_test (3417, 25)
shape of y_test (3417,)


#Define the Artificial Bee Colony algorithm and the model

Model

In [ ]:
# fitness function to Evaluate bees and this function is a SVM model.
# we call SVM to find the best score for classification.
def fitness(c, gamma, x_train, y_train, x_val, y_val):
  #   ------
  #   c         : regularization parameter used to set the tolerance of the model to allow the misclassification of data points in order to achieve lower generalization error. 
  #   gamma     : parameter defines how far the influence of a single training example reaches.
  #   x_train   : array contain data samples for training SVM model.
  #   y_train   : array contain labels for training SVM model.
  #   x_val    : array contain data samples for validating SVM model.
  #   y_val    : array contain labels for validating SVM model.
  # ------

  clf = SVC(C = c, gamma = gamma, kernel ='rbf').fit(x_train, y_train)
  return clf.score(x_val, y_val)


Artificial Bee Colony

In [ ]:
def training(gr_of_bee, gr_of_fit, x_train, y_train, x_val, y_val):
  #   ------
  #   gr_of_bee : list of lists contain pair of C and Gamma which they are the hyperparameter of SVM model. 
  #   gr_of_fit : list contain an accuracy results of SVM model.
  #   x_train   : array contain data samples for training SVM model.
  #   y_train   : array contain labels for training SVM model.
  #   x_test    : array contain data samples for validating SVM model.
  #   y_test    : array contain labels for validating SVM model.
  #   ------

  # Sorting Descending for gr_of_bee according to the sorting Descending for gr_of_fit.
  gr_of_bee = [x for _,x in sorted(zip(gr_of_fit, gr_of_bee),reverse=True)]

  # Sorting Descending for gr_of_fit.
  gr_of_fit.sort(reverse = True)

  A =  0.1   # Neighborhood
  m  = 5     # Random number in range of 0 and the size of gr_of_fit, it used for split the gr_of_bee and gr_of_fit.
  n2 = 3     # Number of bees we have to generate for each bee in E_bee list.
  n1 = 2     # Number of bees we have to generate for each bee in M_E_bee list and must be less than n2.

  # Splitting using m.
  M_bee = gr_of_bee[0:m]
  M_fit = gr_of_fit[0:m]
  N_M_bee = gr_of_bee[m:]
  N_M_fit = gr_of_fit[m:]

  avg = np.array(M_fit).sum()/m    


  E_bee = []
  E_fit = []
  M_E_bee = []
  M_E_fit = []

  # Splitting 'M_fit' and 'M_bee' to 'E_bee' and 'E_fit' if M_fit[i] > avg and 'M_E_bee' and 'M_E_fit' if M_fit[i] < avg.
  for i in range(m):
    if M_fit[i] > avg:
      E_bee.append(M_bee[i])
      E_fit.append(M_fit[i])
    else:
      M_E_bee.append(M_bee[i])
      M_E_fit.append(M_fit[i])
  


  # For each pair in E_bee we generate a 'n2' bee and we pass it to fitness function and compare them with the value of fitness function of this pair  
  # if it's bigger than the value of this pair, we replace it with the bigger value.  
  for count in range(np.array(E_bee).size//2):
    N2_bee = []
    N2_fit = []
    
    for i in range(n2):
      c = random.uniform(E_bee[count][0]-A, E_bee[count][0]+A)
      gamma = random.uniform(E_bee[count][1]-A, E_bee[count][1]+A)

      if c <= 0:
        c = random.uniform(E_bee[count][0], E_bee[count][0]+A)
      
      if gamma <= 0:
        gamma = random.uniform(E_bee[count][1], E_bee[count][1]+A)

      N2_bee.append([c, gamma])

    for c, gamma in N2_bee:
      N2_fit.append(fitness(c, gamma, x_train, y_train, x_val, y_val))

    N2_bee = [x for _,x in sorted(zip(N2_fit, N2_bee),reverse=True)]
    N2_fit.sort(reverse = True) 

    if N2_fit[0] > E_fit[count]:
      E_fit[count] = N2_fit[0]
      E_bee[count] = N2_bee[0]



  # For each pair in M_E_bee we generate a 'n1' bee and we pass it to fitness function and compare them with the value of fitness function of this pair  
  # if it's bigger than the value of this pair, we replace it with the bigger value.
  for count in range(np.array(M_E_bee).size//2):
    N1_bee = []
    N1_fit = []

    for i in range(n1):
      c = random.uniform(M_E_bee[count][0]-A, M_E_bee[count][0]+A)
      gamma = random.uniform(M_E_bee[count][1]-A, M_E_bee[count][1]+A)
      N1_bee.append([c, gamma])

    for c, gamma in N1_bee:
      N1_fit.append(fitness(c, gamma, x_train, y_train, x_val, y_val))
      
    N1_bee = [x for _,x in sorted(zip(N1_fit, N1_bee),reverse=True)]
    N1_fit.sort(reverse = True) 

    if N1_fit[0] > M_E_fit[count]:
      M_E_fit[count] = N1_fit[0]
      M_E_bee[count] = N1_bee[0]



  gr_of_bee = []
  gr_of_fit = []
  N_M_bee = []
  N_M_fit = []

  # Recreate the 'N_M_bee' and 'N_M_fit' by randomly select C and Gamma then pass them to fitness function 
  for i in range(n_bees - m):
    c = random.uniform(2**-5, 2**15)    #  0.001, 10
    gamma = random.uniform(2**-5, 2**15)
    N_M_bee.append([c, gamma])
    N_M_fit.append(fitness(c, gamma, x_train, y_train, x_val, y_val))


  # Lastly, we recreate gr_of_bee by append the 'E_bee', 'M_E_bee', 'N_M_bee' elements to it.
  # and recreate gr_of_fit by append the  'E_fit', 'M_E_fit', 'N_M_fit' elements to it.
  # here the gr_of_bee and gr_of_fit represent the new generation of this iteration.
  for i in range(np.array(E_bee).size//2):
    gr_of_bee.append(E_bee[i])
    gr_of_fit.append(E_fit[i])
    
  for i in range(np.array(M_E_bee).size//2):
    gr_of_bee.append(M_E_bee[i])
    gr_of_fit.append(M_E_fit[i])

  for i in range(np.array(N_M_bee).size//2):
    gr_of_bee.append(N_M_bee[i])
    gr_of_fit.append(N_M_fit[i]) 
  


  # Sorting Descending for gr_of_bee according to the sorting Descending for gr_of_fit.
  gr_of_bee = [x for _,x in sorted(zip(gr_of_fit, gr_of_bee),reverse=True)]
  # Sorting Descending for gr_of_fit.
  gr_of_fit.sort(reverse = True) 



  return gr_of_bee, gr_of_fit
    

# Training

In [ ]:
global_best_score = 0      # Best score for all iteration. 
global_best_bee = []       # Best C and Gamma for all iteration.
n_bees = 10                # Number of bees for the first generation.
gr_of_bee = []
gr_of_fit = []

# Generate c, gamma randomly for bees. 
for i in range(n_bees):
  c = random.uniform(2**-5, 2**15)
  gamma = random.uniform(2**-5, 2**15)
  gr_of_bee.append([c, gamma])

# Calculate the fitness for the genration of c and gamma.
for c, gamma in gr_of_bee:
  gr_of_fit.append(fitness(c, gamma, x_train, y_train, x_val, y_val))

#   ..................... Training .....................

iter = 10
for itr in range(iter):

  bee, fit = training(gr_of_bee, gr_of_fit, x_train, y_train, x_val, y_val)
  gr_of_bee = bee
  gr_of_fit = fit
  bee = []
  fit = []

  if gr_of_fit[0] > global_best_score:
    global_best_score = gr_of_fit[0]
    global_best_bee = gr_of_bee[0]
  print("| iteration: ", itr + 1, " |-------| val_accuracy: ",  gr_of_fit[0], "% ", " |-------| C: ",  gr_of_bee[0][0], " |-------| Gamma: ",  gr_of_bee[0][1], "|")


print()
print("Best val_accuracy find: ", global_best_score, "%")
print("C and Gamma for best accuracy :")
print("C:     ", global_best_bee[0])
print("Gamma: ", global_best_bee[1])

| iteration:  1  |-------| val_accuracy:  0.9992684711046086 %   |-------| C:  5.941419819395952  |-------| Gamma:  0.3039379595170063 |
| iteration:  2  |-------| val_accuracy:  0.9992684711046086 %   |-------| C:  5.941419819395952  |-------| Gamma:  0.3039379595170063 |
| iteration:  3  |-------| val_accuracy:  0.9996342355523044 %   |-------| C:  5.960340782147005  |-------| Gamma:  0.2542448735043387 |
| iteration:  4  |-------| val_accuracy:  1.0 %   |-------| C:  5.967610701582725  |-------| Gamma:  0.21196022744499643 |
| iteration:  5  |-------| val_accuracy:  1.0 %   |-------| C:  5.967610701582725  |-------| Gamma:  0.21196022744499643 |
| iteration:  6  |-------| val_accuracy:  1.0 %   |-------| C:  5.967610701582725  |-------| Gamma:  0.21196022744499643 |
| iteration:  7  |-------| val_accuracy:  1.0 %   |-------| C:  5.967610701582725  |-------| Gamma:  0.21196022744499643 |
| iteration:  8  |-------| val_accuracy:  1.0 %   |-------| C:  5.967610701582725  |-------| Gamm

In [ ]:
# Fit the SVM model on training data.
clf = SVC(C = global_best_bee[0], gamma = global_best_bee[1], kernel ='rbf').fit(x_train, y_train)

# Print Results.
# Confusion matrice
print("Confusion matrice : ")
print(confusion_matrix(y_test, clf.predict(x_test)))

# Accuracy.
clf = SVC(C = global_best_bee[0], gamma = global_best_bee[1], kernel ='rbf').fit(x_train, y_train)
print("Accuracy  : ",clf.score(x_test, y_test))

# Precision.
print("Precision : ",precision_score(y_test, clf.predict(x_test)))

# Recall.
print("Recall    : ",recall_score(y_test, clf.predict(x_test)))


Confusion matrice : 
[[1309    0]
 [   2 2106]]
Accuracy  :  0.9994146912496342
Precision :  1.0
Recall    :  0.9990512333965844
